In [43]:
import modules.singleton_reduction as singleton
import modules.isolate_reduction as isolate
from tqdm import tqdm
import importlib
importlib.reload(singleton)
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor
import io
import copy


In [44]:
# Load the full graph
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)
G_full.add_nodes_from(nodes_array)

## Ego network

In [55]:
# We still compute for the ego network since it takes a fraction of a second to do :)

import modules.ego_layout as ego 
import importlib
importlib.reload(ego)

G = copy.deepcopy(G_full)
G = singleton.singleton_reduction(G)
G_full_sr = copy.deepcopy(G)
G, pos = ego.ego_position(G, ego_target=9)
# No need for isolate reduction, ego network has none by virtue of being an ego network

Node count before singleton reduction:  58742


In [46]:
for i in nx.weakly_connected_components(G):
    print(i)

{3, 26052, 5, 9, 40973, 40978, 16404, 24597, 24601, 16410, 49179, 57371, 24603, 24606, 57374, 49186, 35, 36, 49188, 24614, 24616, 49192, 24619, 49195, 49201, 49208, 24632, 57407, 64, 57409, 63, 24641, 26065, 49222, 16454, 49230, 16463, 41040, 49234, 49235, 57429, 16470, 49237, 49238, 49239, 49241, 49242, 49244, 16477, 24670, 24669, 49246, 49243, 16482, 16483, 24676, 16485, 16486, 16487, 41064, 49252, 49253, 49259, 57452, 49260, 16494, 8303, 16495, 49263, 32882, 115, 32883, 24693, 118, 117, 49266, 49268, 32890, 49274, 57466, 49269, 49283, 41092, 133, 41093, 24711, 136, 57475, 57477, 16532, 148, 41111, 57497, 57501, 32927, 32929, 32930, 32932, 32941, 24749, 49325, 41137, 32949, 32959, 24776, 207, 213, 214, 16600, 41183, 24803, 24804, 41188, 49384, 56018, 33006, 24821, 24833, 33035, 24845, 16655, 49428, 277, 24853, 16663, 289, 49442, 33058, 33059, 33060, 16678, 16674, 57640, 33061, 33062, 16683, 49452, 33063, 33064, 33066, 16691, 57652, 57653, 16696, 24891, 41275, 41276, 57659, 24895, 576

In [74]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
max_steps = 200

G_temp = copy.deepcopy(G)

nodelist_total, nodecolors_total, edgecolors_total, options, infotext_total, constants_total = sir.sir_model(G_temp, G_full_sr, init_infected=[9], infection_rate=0.2, recovery_rate=0.2, max_steps=max_steps, doSingletonReduction=True, noticeability_rates=(0.0001,0.1), network_type='ego', quarantine_length=0)

Running model...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  1
[DEBUG] QT:  0
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  2
[DEBUG] QT:  0
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  3
[DEBUG] QT:  0
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  4
[DEBUG] QT:  1
[INFO] Recovering singletons...
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  5
[DEBUG] QT:  226
[INFO] Recovering singletons...
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  6
[DEBUG] QT:  155
[INFO] Recovering singletons...
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  7
[DEBUG] QT:  109
[INFO] Recovering singletons...
[INFO] Infecting si

In [75]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from functools import partial

# > Clear all pngs in graphs folder
for filename in os.listdir("graphs"):
   file_path = os.path.join("graphs", filename)
   if os.path.isfile(file_path):
      os.remove(file_path)

worker = partial(
    sir.work,
    G=G,
    pos=pos,
    nodelist_total=nodelist_total,
    nodecolors_total=nodecolors_total,
    edgecolors_total=edgecolors_total,
    infotext_total=infotext_total,
    constants_total=constants_total,
    options=options,
    doDrawEdges=False
)

with Pool(8) as p:
    p.map(worker, range(len(nodelist_total)))

Starting graph image: 2
Starting graph image: 4
Starting graph image: 6
Starting graph image: 8
Starting graph image: 0
Starting graph image: 10
Starting graph image: 12
Starting graph image: 14


/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a 

Saved graph_0.png!
Starting graph image: 1
Saved graph_2.png!
Starting graph image: 3
Saved graph_8.png!
Starting graph image: 9
Saved graph_4.png!
Starting graph image: 5
Saved graph_6.png!
Starting graph image: 7
Saved graph_10.png!
Starting graph image: 11
Saved graph_12.png!
Starting graph image: 13
Saved graph_1.png!
Starting graph image: 16


/opt/anaconda3/lib/python3.12/site-packages/networkx/drawing/nx_pylab.py:1495: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(


Saved graph_14.png!
Starting graph image: 15
Saved graph_3.png!
Saved graph_5.png!
Starting graph image: 18
Saved graph_9.png!
Saved graph_7.png!
Saved graph_11.png!
Starting graph image: 20
Saved graph_13.png!
Starting graph image: 22
Saved graph_16.png!
Starting graph image: 17
Saved graph_15.png!
Starting graph image: 24
Starting graph image: 26
Saved graph_18.png!
Starting graph image: 19
Starting graph image: 28
Saved graph_20.png!
Starting graph image: 21
Saved graph_22.png!
Starting graph image: 23
Saved graph_17.png!
Starting graph image: 30
Saved graph_24.png!
Starting graph image: 25
Saved graph_26.png!
Starting graph image: 27
Starting graph image: 32
Saved graph_19.png!
Saved graph_23.png!
Starting graph image: 34
Saved graph_28.png!
Starting graph image: 29
Starting graph image: 36
Saved graph_21.png!
Starting graph image: 38
Saved graph_30.png!
Starting graph image: 31
Saved graph_25.png!
Starting graph image: 40
Saved graph_32.png!
Starting graph image: 33
Saved graph_27

## Exporting as GIF

In [73]:
# > Export images in graphs folder to a gif
from PIL import Image

# List of image file paths
images = sorted(os.listdir("graphs"), key=lambda x:int(x[x.find('_')+1:x.find('.')]))

# Open images and store them in a list
frames = [Image.open(f"graphs/{image}") for image in images]

# Save frames as an animated GIF
frames[0].save(
    'gifs/sir_model_ego.gif',
    save_all=True,
    append_images=frames[1:],
    duration=500,
    loop=0
)

## Finding number of nodes and which nodes needed to infect the whole network

In [89]:
from modules.number_of_parent_nodes import parent_node_number
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]
# With isolated nodes (no in or out degree)
infected_nodes_list_isolated = parent_node_number(edges_array, nodes_array, includeIsolatedNodes=True)
# Gathering number of nodes
print(len(infected_nodes_list_isolated))
# List of nodes
print(infected_nodes_list_isolated)

# Without isolated nodes
infected_nodes_list_no_isolated = parent_node_number(edges_array, nodes_array, includeIsolatedNodes=False)
# Gathering number of nodes
print(len(infected_nodes_list_no_isolated))
# List of nodes
print(infected_nodes_list_no_isolated)



38221
[np.int64(169), np.int64(594), np.int64(81), np.int64(595), np.int64(131), np.int64(10), np.int64(596), np.int64(384), np.int64(660), np.int64(657), np.int64(661), np.int64(665), np.int64(666), np.int64(755), np.int64(15), np.int64(1091), np.int64(69), np.int64(342), np.int64(119), np.int64(160), np.int64(1766), np.int64(981), np.int64(111), np.int64(299), np.int64(817), np.int64(2462), np.int64(2463), np.int64(137), np.int64(1158), np.int64(261), np.int64(2464), np.int64(121), np.int64(147), np.int64(816), np.int64(231), np.int64(2871), np.int64(3063), np.int64(3064), np.int64(199), np.int64(3168), np.int64(3957), np.int64(892), np.int64(6), np.int64(3169), np.int64(3170), np.int64(336), np.int64(74), np.int64(57), np.int64(705), np.int64(1707), np.int64(2658), np.int64(1651), np.int64(2117), np.int64(2122), np.int64(7046), np.int64(7047), np.int64(1472), np.int64(821), np.int64(2155), np.int64(959), np.int64(2366), np.int64(7770), np.int64(2852), np.int64(1084), np.int64(7774),

In [ ]:
# TODO: need to add all infected nodes to simulation and run the simulation once to showcase infection of all nodes in network (this can be done at a later date right now im tired)